In [15]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kaggle
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [3]:
## Get Dataset from Kaggle

import kaggle
# Requires kaggle.json (Kaggle API Key) placed in ~/.kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files("drgilermo/nba-players-stats", path="./data/", unzip=True)

In [8]:
season_data = pd.read_csv("./season_stats_separated_pos.csv")
season_data.head(1)

,Unnamed: 0,Index,Year,Player,Pos,Age,Tm,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,0,1950.0,Curly Armstrong,G,31.0,FTW,63.0,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0


In [10]:
season_data.shape

(14755, 49)

In [9]:
# Clean Data by dropping columns with all NaN values
season_data.drop(['Unnamed: 0', 'blanl', 'blank2'], axis=1, inplace=True)
season_data.dropna(inplace=True)

# Also dropping the name of the player...hopefully players aren't assigned positions based on their names
# Also dropping the year since we are not predicting the position of a player in a specific year
season_data.drop(['Year', 'Player'], axis=1, inplace=True)
season_data.head(5)

,Index,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5730,5730,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,0.023,...,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5748,5748,PF,23.0,BOS,82.0,82.0,2955.0,20.5,0.538,0.098,...,0.836,216.0,636.0,852.0,370.0,143.0,53.0,263.0,279.0,1745.0
5781,5781,SF,29.0,BOS,82.0,7.0,1994.0,14.6,0.526,0.054,...,0.739,106.0,224.0,330.0,156.0,120.0,36.0,143.0,214.0,914.0
5786,5786,SG,33.0,BOS,60.0,0.0,523.0,7.8,0.402,0.032,...,0.762,31.0,42.0,73.0,38.0,31.0,11.0,33.0,80.0,167.0
5802,5802,C,31.0,BOS,66.0,55.0,2159.0,14.8,0.477,0.013,...,0.779,126.0,408.0,534.0,206.0,69.0,61.0,108.0,216.0,940.0


In [16]:
le = LabelEncoder()
y = season_data['Pos']
y = le.fit_transform(y)
X = season_data.drop('Pos', axis=1)
X['Tm'] = le.fit_transform(X['Tm']) # Encode team names as numerical labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='multi:softmax')
bst.fit(X_train, y_train)
preds = bst.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(f"Accuracy: {accuracy*100}%")

Accuracy: 61.3690274483226%


In [17]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [18]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100}%")

Accuracy: 67.67197560149101%


In [19]:
y = season_data['Pos']
y = le.fit_transform(y)
X = season_data[['AST%', 'TRB%', 'DRB%', 'ORB%', 'BLK%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5967468654693324
